# Practical 6.2

# Word-level Sequence-to-Sequence (Seq2Seq) Model

# Application-1: Machine Translation  

Similar architecture and objectives with Practical 6.2, but we will train the translation model on word sequences.

![Image](rnn_word_translation.png?raw=true)

In [3]:
from __future__ import print_function

import os
import sys
import numpy as np
import nltk
import string
from string import punctuation
import re

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, GRU, Lambda, Bidirectional, concatenate

Using TensorFlow backend.


In [ ]:
import gensim
from gensim.models import Word2Vec

## 1. Data preprocessing

In [0]:
file_to_read = 'nld.txt'

Function to tokenize text into words (array list of words). Notice that we discard all punctuation in original text.

In [0]:
def tokenizeWords(text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    clean_text = regex.sub('', text)
    tokens = clean_text.split()
   
       
    return [t.lower() for t in tokens]

In [0]:
def indexingVocabulary(array_of_words):
    
    # frequency of word across document corpus
    tf = nltk.FreqDist(array_of_words)
    wordIndex = list(tf.keys())
    
    wordIndex.insert(0,'<pad>')
    wordIndex.append('<start>')
    wordIndex.append('<end>')
    wordIndex.append('<unk>')
    # indexing word vocabulary : pairs of (index,word)
    vocab=dict([(i,wordIndex[i]) for i in range(len(wordIndex))])
    
    return vocab

In [0]:
#reading text line by line
lines = open(os.path.join(local_download_path,file_to_read)).read().split('\n')

### Tokenization and vocabulary indexing

Notice that we only use 10.000 samples from data. Training a Seq2Seq model is computationally expensive (memory!)

In [0]:
num_samples = 10000  # Number of samples to train on.

input_str_tokens = []
target_str_tokens = []

ind_start = 10000
ind_end = 10000 + min(num_samples, len(lines) - 1)

for line in lines[ind_start : ind_end]:
    input_text, target_text = line.split('\t')
    # tokenize text from source language (english text)
    input_str_tokens.append(tokenizeWords(input_text))
    # tokenize text from target language (dutch text)
    target_str_tokens.append(tokenizeWords(target_text))

* `en_vocab` stores word index for encoder input sequences (english dictionary)
* `nl_vocab` stores word index for decoder target sequences (dutch dictionary)

In [0]:
# build vocabulary index
input_words = []
target_words = []

for i, tokens in enumerate(input_str_tokens):
    input_words.extend(tokens)
# vocabulary index for english text (input)    
en_vocab = indexingVocabulary(input_words)

for i, tokens in enumerate(target_str_tokens):
    target_words.extend(tokens)
# vocabulary index for dutch text (output)
nl_vocab = indexingVocabulary(target_words)

We also need to create reverse version of look up index to map text sequences into integer format.

In [0]:
en_reversedvocab = dict((v,k) for (k,v) in en_vocab.items())
nl_reversedvocab = dict((v,k) for (k,v) in nl_vocab.items())

### Preparing training sequences


* `seq_int_input`: input sequences for encoder model 
* `seq_int_target`: input sequences for decoder model

In [0]:
# integer format of sequence input 
seq_int_input = []
for i, text in enumerate(input_str_tokens):
    int_tokens = [en_reversedvocab[i] for i in text]
    seq_int_input.append(int_tokens)

For input and output sequences of decoder model, we add `starting` sign (`'<start>'`) and `ending` sign (`'<end>'`) at the beginning and last part of sequence. 

In [0]:
seq_int_target = []
for i, text in enumerate(target_str_tokens):
    targettext = list(text)
    targettext.insert(0,'<start>')
    targettext.append('<end>')
  
    int_tokens = [nl_reversedvocab[i] for i in targettext]
    seq_int_target.append(int_tokens)

## 2. Word embedding

* In character level, we define input and output sequences as one-hot vector in 3D numpy arrays (number of samples, sequence length, vocabulary size). 
* For word-level, we have integer input sequences with 2D shape (number of samples, sequence length). Instead of one-hot encoding words, we will use embedding layer to project each word sequence to its embedding.
* We will train our text with Word2Vec - Skipgram to provide initial weight for our embedding layer (may also use pretrained word embedding).

In [0]:
# for english text
# skipgram model with hierarchical softmax and negative sampling
word2vec_model_en = Word2Vec(size=256, min_count=0, window=5, sg=1, 
                          hs=1, negative=5, iter=100)

In [0]:
word2vec_model_en.build_vocab(input_str_tokens)
word2vec_vocab_en = dict([(v.index,k) for k, v in word2vec_model_en.wv.vocab.items()]) 
revert_w2v_vocab_en = dict((v,k) for (k,v) in word2vec_vocab_en.items())

In [0]:
# for dutch text
# skipgram model with hierarchical softmax and negative sampling
word2vec_model_nl = Word2Vec(size=256, min_count=0, window=5, sg=1, 
                          hs=1, negative=5, iter=100)

In [0]:
word2vec_model_nl.build_vocab(target_str_tokens)
word2vec_vocab_nl = dict([(v.index,k) for k, v in word2vec_model_nl.wv.vocab.items()]) 
revert_w2v_vocab_nl = dict((v,k) for (k,v) in word2vec_vocab_nl.items())

In [30]:
print('Training word2vec model...')

# for english text
# number of tokens
n_tokens = sum([len(seq) for seq in input_str_tokens])
# number of sentences/documents
n_examples = len(input_str_tokens)
word2vec_model_en.train(input_str_tokens, total_words=n_tokens, 
                        total_examples=n_examples, epochs=100)

Training word2vec model...


(4257792, 5946200)

In [31]:
# for dutch text
# number of tokens
n_tokens = sum([len(seq) for seq in target_str_tokens])
# number of sentences/documents
n_examples = len(target_str_tokens)
word2vec_model_nl.train(target_str_tokens, total_words=n_tokens, 
                        total_examples=n_examples, epochs=100)

(4297470, 6123900)

The following variables store our word embedding learnt from word2vec skipgram.

In [ ]:
# the resulting learnt word embedding 
# for input text sequence (english language)
word2vec_we_en = word2vec_model_en.wv.syn0

# for target text sequence (dutch language)
word2vec_we_nl = word2vec_model_nl.wv.syn0

Notice that vocabulary size of word embedding learnt by word2vec is less than our vocabulary size since we add additional word tokens: `'<pad>'`, `'<start>'`, `'<end>'`, `'<unk>'`

In [33]:
word2vec_we_en.shape

(4163, 256)

In [34]:
word2vec_we_nl.shape

(5280, 256)

In [0]:
embedding_en = np.zeros(shape=(len(en_vocab), 256), dtype='float32')
embedding_nl = np.zeros(shape=(len(nl_vocab), 256), dtype='float32')

In [41]:
embedding_en.shape

(4167, 256)

In [42]:
embedding_nl.shape

(5284, 256)

In [0]:
# for input sequences (text in english language)
for i, w in en_vocab.items():
    # this will assign default weight 0 for words: 'SOF', 'EOF', and 'UNK'
    if w not in word2vec_vocab_en.values():
        continue
    embedding_en[i, :] = word2vec_we_en[revert_w2v_vocab_en[w], :]

In [0]:
# for target output sequences (text in dutch language)
for i, w in nl_vocab.items():
    # this will assign default weight 0 for words: 'SOF', 'EOF', and 'UNK'
    if w not in word2vec_vocab_nl.values():
        continue
    embedding_nl[i, :] = word2vec_we_nl[revert_w2v_vocab_nl[w], :]

## 3. Word-based Translation model

Parameters initialization.

In [0]:
batch_size = 100  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
#num_samples = 1000  # Number of samples to train on.

EN_VOCAB_SIZE = len(en_vocab) # vocabulary size of english words
EN_EMBEDDING_DIM = embedding_en.shape[1] # embedding size of english words
NL_VOCAB_SIZE = len(nl_vocab) # vocabulary size of dutch words
NL_EMBEDDING_DIM = embedding_nl.shape[1] # embedding size of dutch words

## 3.1. Encoder model

Notice that in this example, we set parameters in our embedding layer to be trainable. 

In [0]:
# encoder model
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(EN_VOCAB_SIZE, EN_EMBEDDING_DIM, trainable = True, 
              weights=[embedding_en], name='embedding_encoder')(encoder_inputs)
lstm_encoder, state_h, state_c = LSTM(latent_dim, return_state=True, name='lstm_encoder')(encoder_embedding)
encoder_states = [state_h, state_c]

## 3.2. Decoder model

In [0]:
# decoder model
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_embedding = Embedding(NL_VOCAB_SIZE, NL_EMBEDDING_DIM, trainable = True, 
              weights=[embedding_nl], name='embedding_decoder')
embedding_output = decoder_embedding(decoder_inputs)
lstm_decoder = LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_decoder')
decoder_outputs, _, _  = lstm_decoder(embedding_output, initial_state=encoder_states)

decoder_dense = Dense(NL_VOCAB_SIZE, activation='softmax', name='prediction_layer')
prediction_outputs = decoder_dense(decoder_outputs)


In [0]:
model = Model([encoder_inputs, decoder_inputs], prediction_outputs)

In [0]:
# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [50]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
embedding_encoder (Embedding)   (None, None, 256)    1066752     encoder_inputs[0][0]             
__________________________________________________________________________________________________
embedding_decoder (Embedding)   (None, None, 256)    1352704     decoder_inputs[0][0]             
__________________________________________________________________________________________________
lstm_encod

## Training translation model

In [0]:
max_encoder_seq_length = max([len(sequences) for sequences in seq_int_input])
max_decoder_seq_length = max([len(sequences) for sequences in seq_int_target])

In [52]:
max_encoder_seq_length

10

In [53]:
max_decoder_seq_length

16

In [0]:
decoder_target_data = np.zeros((len(seq_int_input), max_decoder_seq_length, NL_VOCAB_SIZE),dtype='float32')

In [0]:
encoder_input_data = np.zeros((len(seq_int_input), max_encoder_seq_length), dtype='float32')

In [0]:
decoder_input_data = np.zeros((len(seq_int_input), max_decoder_seq_length), dtype='float32')

In [57]:
encoder_input_data.shape

(10000, 10)

In [58]:
decoder_input_data.shape

(10000, 16)

In [59]:
decoder_target_data.shape

(10000, 16, 5284)

Be aware that the output of decoder layer need to be in categorical format. 

In [0]:
def one_hot_vectorization(seqs,seq_length,vocab_size):
    onehotsequences = np.zeros((len(seqs), seq_length, vocab_size))
    for i, text in enumerate(seqs):
        for j, word_index in enumerate(text):
            onehotsequences[i, j, int(word_index)] = 1
    return onehotsequences
   

### Padding input sequences for encoder and decoder

In [0]:
for i, seq_int in enumerate(seq_int_input):
    for j, word_index in enumerate(seq_int):
        encoder_input_data[i][j] = word_index

In [0]:
for i, seq_int in enumerate(seq_int_target):
    for j, word_index in enumerate(seq_int):
        decoder_input_data[i][j] = word_index

In [68]:
decoder_input_data.shape

(10000, 16)

In [0]:
decoder_target_data.shape

(1000, 10, 880)

### Fitting sequences into model

* Note: creating 3D numpy arrays of decoder output (one-hot-encoding) might cause Memory Error. 


In [0]:
k_start = 1 
i_end = 0
mini_batches = 50

In [76]:
for k in range(k_start, epochs+1):
    for i in range(0, len(encoder_input_data), mini_batches):
        if i + mini_batches >= len(encoder_input_data):
            i_end = len(encoder_input_data)
        else:
            i_end = i + mini_batches
        # transform padded X_decoder into one-hot vector
        decoder_target_data = one_hot_vectorization(decoder_input_data[i:i_end],max_decoder_seq_length,NL_VOCAB_SIZE)
        dec_target_data = decoder_target_data[:,1:]
        dec_input_data = decoder_input_data[i:i_end,0:-1]
        enc_input_data = encoder_input_data[i:i_end]
        
        print('[INFO] Training model: epoch {}th {}/{} samples'. format(k,i,len(encoder_input_data)))
        model.fit([enc_input_data, dec_input_data], dec_target_data, 
                  batch_size=batch_size, epochs=1, verbose=2, validation_split=0.2)
            

[INFO] Training model: epoch 1th 0/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0097 - val_loss: 1.7561
[INFO] Training model: epoch 1th 50/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0015 - val_loss: 2.1553
[INFO] Training model: epoch 1th 100/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0081 - val_loss: 2.0809
[INFO] Training model: epoch 1th 150/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.8132 - val_loss: 1.9375
[INFO] Training model: epoch 1th 200/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.7242 - val_loss: 2.0783
[INFO] Training model: epoch 1th 250/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.9710 - val_loss: 1.7845
[INFO] Training model: epoch 1th 300/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0143 - val_l

[INFO] Training model: epoch 1th 4400/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.2536 - val_loss: 2.0148
[INFO] Training model: epoch 1th 4450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.3417 - val_loss: 2.4661
[INFO] Training model: epoch 1th 4500/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.4605 - val_loss: 2.5606
[INFO] Training model: epoch 1th 4550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.3623 - val_loss: 2.3442
[INFO] Training model: epoch 1th 4600/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.3060 - val_loss: 2.3758
[INFO] Training model: epoch 1th 4650/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.3235 - val_loss: 2.2133
[INFO] Training model: epoch 1th 4700/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.32

[INFO] Training model: epoch 1th 8550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.4919 - val_loss: 2.3839
[INFO] Training model: epoch 1th 8600/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.7319 - val_loss: 2.1712
[INFO] Training model: epoch 1th 8650/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.3489 - val_loss: 2.3562
[INFO] Training model: epoch 1th 8700/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.6662 - val_loss: 2.3000
[INFO] Training model: epoch 1th 8750/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.4298 - val_loss: 1.9772
[INFO] Training model: epoch 1th 8800/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.4713 - val_loss: 2.6054
[INFO] Training model: epoch 1th 8850/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.44

[INFO] Training model: epoch 2th 2700/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.9739 - val_loss: 2.1001
[INFO] Training model: epoch 2th 2750/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.8186 - val_loss: 1.7665
[INFO] Training model: epoch 2th 2800/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.6941 - val_loss: 1.5765
[INFO] Training model: epoch 2th 2850/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0304 - val_loss: 2.0348
[INFO] Training model: epoch 2th 2900/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.6760 - val_loss: 1.6438
[INFO] Training model: epoch 2th 2950/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.7581 - val_loss: 2.0481
[INFO] Training model: epoch 2th 3000/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.83

[INFO] Training model: epoch 2th 6800/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.1556 - val_loss: 1.8699
[INFO] Training model: epoch 2th 6850/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 1.8935 - val_loss: 2.0414
[INFO] Training model: epoch 2th 6900/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0491 - val_loss: 2.3641
[INFO] Training model: epoch 2th 6950/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.1449 - val_loss: 1.9992
[INFO] Training model: epoch 2th 7000/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0368 - val_loss: 2.3203
[INFO] Training model: epoch 2th 7050/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.0908 - val_loss: 2.3194
[INFO] Training model: epoch 2th 7100/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 2.17

...

[INFO] Training model: epoch 99th 3250/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0583 - val_loss: 0.9328
[INFO] Training model: epoch 99th 3300/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0374 - val_loss: 1.3193
[INFO] Training model: epoch 99th 3350/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0663 - val_loss: 2.0521
[INFO] Training model: epoch 99th 3400/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0445 - val_loss: 1.2618
[INFO] Training model: epoch 99th 3450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0663 - val_loss: 1.2303
[INFO] Training model: epoch 99th 3500/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0559 - val_loss: 1.7961
[INFO] Training model: epoch 99th 3550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - los

Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0454 - val_loss: 1.7636
[INFO] Training model: epoch 99th 7450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0639 - val_loss: 2.2370
[INFO] Training model: epoch 99th 7500/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0993 - val_loss: 2.0272
[INFO] Training model: epoch 99th 7550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0689 - val_loss: 1.3530
[INFO] Training model: epoch 99th 7600/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0678 - val_loss: 2.6678
[INFO] Training model: epoch 99th 7650/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0625 - val_loss: 1.6607
[INFO] Training model: epoch 99th 7700/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0536 - val_loss: 2.0636
[INFO] Training model: e

[INFO] Training model: epoch 100th 1300/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0382 - val_loss: 1.2280
[INFO] Training model: epoch 100th 1350/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0485 - val_loss: 1.4427
[INFO] Training model: epoch 100th 1400/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0404 - val_loss: 1.6556
[INFO] Training model: epoch 100th 1450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0481 - val_loss: 1.8788
[INFO] Training model: epoch 100th 1500/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0412 - val_loss: 1.7026
[INFO] Training model: epoch 100th 1550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0543 - val_loss: 1.3269
[INFO] Training model: epoch 100th 1600/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0

[INFO] Training model: epoch 100th 5300/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0378 - val_loss: 1.7772
[INFO] Training model: epoch 100th 5350/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0382 - val_loss: 1.2853
[INFO] Training model: epoch 100th 5400/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0433 - val_loss: 1.5321
[INFO] Training model: epoch 100th 5450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0463 - val_loss: 2.3704
[INFO] Training model: epoch 100th 5500/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0599 - val_loss: 0.6524
[INFO] Training model: epoch 100th 5550/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0414 - val_loss: 2.0149
[INFO] Training model: epoch 100th 5600/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0

 - 0s - loss: 0.0732 - val_loss: 1.5193
[INFO] Training model: epoch 100th 9200/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0907 - val_loss: 1.6347
[INFO] Training model: epoch 100th 9250/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0653 - val_loss: 1.4066
[INFO] Training model: epoch 100th 9300/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0983 - val_loss: 2.4837
[INFO] Training model: epoch 100th 9350/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0864 - val_loss: 2.5208
[INFO] Training model: epoch 100th 9400/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.1125 - val_loss: 2.4583
[INFO] Training model: epoch 100th 9450/10000 samples
Train on 40 samples, validate on 10 samples
Epoch 1/1
 - 0s - loss: 0.0825 - val_loss: 1.9313
[INFO] Training model: epoch 100th 9500/10000 samples
Train on 40 sample

In [ ]:
# Save model
model.save('rnn_word_lstm_translation.h5')

In [0]:
model.save_weights('weights_rnn_word_lstm_translation.hdf5')

## 4. Inference mode

## 4.1. Re-define encoder model

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

In [0]:
encoder_model.save('encoder_word_lstm_translation.h5')

## 4.2. Re-define decoder model to do the inference

In [0]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
embedding_dec =  decoder_embedding(decoder_inputs)
decoder, state_h, state_c= lstm_decoder(embedding_dec, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
decoder_model.save(os.path.join('decoder_word_lstm_translation.h5')

## 4.3. Translate sentence

In [0]:
def decode_sequence(input_seq,dec_input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
      
    # Populate the first character of target sequence with the start character.
    target_seq = np.zeros((1, 1))
    target_seq[0][0] = dec_input_seq[0][0]
    
    stop_condition = False
    decoded_sentence = ''
    decoded_int = []
    sent = []
    cnt_token = 0
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = nl_vocab[sampled_token_index]
        cnt_token += 1
        decoded_int.append(sampled_token_index)
        if(sampled_word != '<pad>') and (sampled_word != '<end>'):
            sent.append(sampled_word)
           
        # Exit condition: hit max length
        if(cnt_token > max_decoder_seq_length):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0][0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
        
    decoded_sentence = ' '.join(sent)  

    return decoded_int,decoded_sentence

In [94]:
for seq_index in range(10):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    target_seq = decoder_input_data[seq_index: seq_index + 1]
       
    decoded_int, decoded_sentence = decode_sequence(input_seq,target_seq)
    print('---')
    print('Input sentence:', input_str_tokens[seq_index])
    print('Original output sentence:', target_str_tokens[seq_index])
    print('Output of translation model: ')
    print('Decoded int:', decoded_int)
    print('Decoded sentence:', decoded_sentence)
   

---
Input sentence: ['good', 'students', 'study', 'hard']
Original output sentence: ['goede', 'studenten', 'studeren', 'hard']
Output of translation model: 
Decoded int: [1, 2, 3, 4, 5282, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded sentence: goede studenten studeren hard
---
Input sentence: ['greece', 'is', 'an', 'old', 'country']
Original output sentence: ['griekenland', 'is', 'een', 'oud', 'land']
Output of translation model: 
Decoded int: [7, 8, 6, 8, 9, 5282, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded sentence: een oud is oud land
---
Input sentence: ['have', 'any', 'of', 'you', 'seen', 'tom']
Original output sentence: ['heeft', 'iemand', 'van', 'jullie', 'tom', 'gezien']
Output of translation model: 
Decoded int: [10, 11, 12, 13, 14, 15, 5282, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded sentence: heeft iemand van jullie tom gezien
---
Input sentence: ['have', 'you', 'been', 'practicing']
Original output sentence: ['heb', 'je', 'geoefend']
Output of translation model: 
Decoded int: [16, 1